<a href="https://colab.research.google.com/github/OleksiiLatypov/Bot_Assistant/blob/master/Kaggle/Ghouls_Goblins_and_Ghosts_Boo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ghouls-goblins-and-ghosts-boo:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F5708%2F39544%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240208%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240208T202329Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D40e08af5c8a8d2b8d23cecc90f8341b4c443f5fd00b04e9f1d4c752ca8b8acdccabd1474bc903d08e2e00359ebfd5e4c92808dbe10a2c8ac80dec7127705e874b6465b36934c32b17e977a151183a8bb00b49126b10c87352fa32fd5befee90c65d36a29b0f42405bb1c004232ea10c09696231193b936ca9907475a88fd0cf1ccdcb9114d68dd1adc3e865b827cc1ace372bf0349ac481869d1ba9dc67c59c0a81dc853c3470ef37c30989e649c94a5318f03d48ded78197daf3e539955cd366ea06ad1479fcd008193de1d894c112c4b7d5f24f3d422237cf460a592fd484fb86ebd83f8e9e3f5b8cecb08ef54f14c7f044d107326e04f733ed1191e50537d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 38895 bytes downloaded
Downloaded and uncompressed: ghouls-goblins-and-ghosts-boo
Data source import complete.


In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip
/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip
/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip


In [48]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

In [49]:
train = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip")
test = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip")

In [50]:
train

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost
...,...,...,...,...,...,...,...
366,886,0.458132,0.391760,0.660590,0.635689,blue,Goblin
367,889,0.331936,0.564836,0.539216,0.551471,green,Ghost
368,890,0.481640,0.501147,0.496446,0.544003,clear,Ghoul
369,896,0.294943,0.771286,0.583503,0.300618,clear,Ghost


In [51]:
test

,id,bone_length,rotting_flesh,hair_length,has_soul,color
0,3,0.471774,0.387937,0.706087,0.698537,black
1,6,0.427332,0.645024,0.565558,0.451462,white
2,9,0.549602,0.491931,0.660387,0.449809,black
3,10,0.638095,0.682867,0.471409,0.356924,white
4,13,0.361762,0.583997,0.377256,0.276364,black
...,...,...,...,...,...,...
524,893,0.377573,0.390158,0.696465,0.355373,blue
525,894,0.229161,0.601265,0.191282,0.475115,clear
526,895,0.510497,0.498347,0.708020,0.714154,white
527,898,0.331472,0.765835,0.338207,0.193431,clear


In [52]:
saam = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip", low_memory = False)
saam.head(20)

,id,type
0,3,Ghost
1,6,Ghost
2,9,Ghost
3,10,Ghost
4,13,Ghost
5,14,Ghost
6,15,Ghost
7,16,Ghost
8,17,Ghost
9,18,Ghost


In [53]:
train['type'].unique()

array(['Ghoul', 'Goblin', 'Ghost'], dtype=object)

In [54]:
train['color'].value_counts()

white    137
clear    120
green     42
black     41
blue      19
blood     12
Name: color, dtype: int64

In [55]:
train['type'].value_counts()

Ghoul     129
Goblin    125
Ghost     117
Name: type, dtype: int64

In [56]:
train.describe(include='all')

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
count,371.000000,371.000000,371.000000,371.000000,371.000000,371,371
unique,NaN,NaN,NaN,NaN,NaN,6,3
top,NaN,NaN,NaN,NaN,NaN,white,Ghoul
freq,NaN,NaN,NaN,NaN,NaN,137,129
mean,443.676550,0.434160,0.506848,0.529114,0.471392,NaN,NaN
std,263.222489,0.132833,0.146358,0.169902,0.176129,NaN,NaN
min,0.000000,0.061032,0.095687,0.134600,0.009402,NaN,NaN
25%,205.500000,0.340006,0.414812,0.407428,0.348002,NaN,NaN
50%,458.000000,0.434891,0.501552,0.538642,0.466372,NaN,NaN
75%,678.500000,0.517223,0.603977,0.647244,0.600610,NaN,NaN


In [57]:
test.describe(include='all')

,id,bone_length,rotting_flesh,hair_length,has_soul,color
count,529.000000,529.000000,529.000000,529.000000,529.000000,529
unique,NaN,NaN,NaN,NaN,NaN,6
top,NaN,NaN,NaN,NaN,NaN,white
freq,NaN,NaN,NaN,NaN,NaN,197
mean,453.584121,0.425474,0.503637,0.517288,0.464096,NaN
std,257.804948,0.136079,0.158552,0.171727,0.179659,NaN
min,3.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,239.000000,0.329383,0.393987,0.388189,0.338964,NaN
50%,445.000000,0.421989,0.510065,0.520220,0.461294,NaN
75%,670.000000,0.519024,0.605957,0.644824,0.584448,NaN


In [58]:
y_train=train['type']
index_train=train['id']
index_test=test['id']
train=train.drop(['id','type'],axis=1)
test=test.drop(['id'],axis=1)

In [59]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [60]:
train.head()

,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,0.354512,0.350839,0.465761,0.781142,0,0,0,1,0,0
1,0.575560,0.425868,0.531401,0.439899,0,0,0,0,1,0
2,0.467875,0.354330,0.811616,0.791225,1,0,0,0,0,0
3,0.776652,0.508723,0.636766,0.884464,1,0,0,0,0,0
4,0.566117,0.875862,0.418594,0.636438,0,0,0,0,1,0


In [61]:
X_train,X_validation,Y_train,Y_validation=train_test_split(train,y_train,test_size=0.2,random_state=0)

In [62]:
#Logistic Regression
lr=LogisticRegression()
lr.fit(X_train,Y_train)
y_validation_pred=lr.predict(X_validation)
print(accuracy_score(y_validation_pred,Y_validation))

0.8


In [63]:
sc=StandardScaler()
X_train_std=sc.fit_transform(X_train)
X_validation_std=sc.transform(X_validation)

In [64]:
lr=LogisticRegression(C=0.1, penalty='l2', max_iter=1000, class_weight='balanced')
lr.fit(X_train_std,Y_train)
y_validation_pred=lr.predict(X_validation_std)

print(classification_report(Y_validation, y_validation_pred))
accuracy = accuracy_score(Y_validation, y_validation_pred)
print("Accuracy Score:", accuracy)

              precision    recall  f1-score   support

       Ghost       0.93      0.93      0.93        29
       Ghoul       0.86      0.82      0.84        22
      Goblin       0.76      0.79      0.78        24

    accuracy                           0.85        75
   macro avg       0.85      0.85      0.85        75
weighted avg       0.85      0.85      0.85        75

Accuracy Score: 0.8533333333333334


In [65]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(Y_train)
y_validation_encoded = label_encoder.transform(Y_validation)

# Create an instance of XGBClassifier
xgb = XGBClassifier()

# Fit the model on the training data
xgb.fit(X_train, y_train_encoded)

# Make predictions on the validation set
y_validation_pred_encoded = xgb.predict(X_validation)

# Inverse transform the encoded predictions to get original labels
y_validation_pred = label_encoder.inverse_transform(y_validation_pred_encoded)

# Evaluate the accuracy
accuracy = accuracy_score(Y_validation, y_validation_pred)
print('Accuracy:', accuracy)

Accuracy: 0.72


In [66]:
train_std=sc.fit_transform(train)
test_std=sc.transform(test)

lr.fit(train_std,y_train)
y_pred=lr.predict(test_std)

In [67]:
submit1=pd.DataFrame()
submit1['id']=index_test
submit1['type']=y_pred
submit1
submit1.to_csv("no_index.csv",sep=',',header=True,index=False)

In [69]:
text = pd.read_csv('/content/no_index.csv')
text

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Ghost
4,13,Ghost
...,...,...
524,893,Goblin
525,894,Ghost
526,895,Ghoul
527,898,Ghost


In [70]:
# pred = rf_classifier.predict(test)
# print(pred)

In [71]:
# saam['type'] = pred
# saam.to_csv('pred.csv', index = False)